In [17]:
from langchain_community.document_loaders import PyMuPDFLoader

In [1]:
%cd "C:\Users\PUNEETH\Desktop\All files\RAG\niyathi_rag"

C:\Users\PUNEETH\Desktop\All files\RAG\niyathi_rag


Extracting text from Docs

In [19]:
from langchain_community.document_loaders import PyMuPDFLoader
from src.backend.textextractor import DocumentLoader

In [20]:
texts = DocumentLoader(path=r"src\docs").load()

Loading all files from folder: src\docs
 → Loading: src\docs\doc1.pdf
Loading file: src\docs\doc1.pdf
 → Loading: src\docs\doc2.pdf
Loading file: src\docs\doc2.pdf

✅ Total documents loaded: 60


In [21]:
texts[59]

Document(metadata={'producer': 'Acrobat Distiller 6.0 for Windows', 'creator': 'Elsevier', 'creationdate': '2008-12-01T13:19:14+05:30', 'source': 'src\\docs\\doc2.pdf', 'file_path': 'src\\docs\\doc2.pdf', 'total_pages': 38, 'format': 'PDF 1.4', 'title': 'Essentials of Cardiology', 'author': 'Timothy C. Slesnick; Ralph Gertler; Wanda C. Miller-Hance', 'subject': '', 'keywords': '', 'moddate': '2009-01-05T18:26:04+05:30', 'trapped': '', 'modDate': "D:20090105182604+05'30'", 'creationDate': "D:20081201131914+05'30'", 'page': 37}, page_content='330\nresult in a normal life expectancy and normal cardiovascular \nreserve.332 Th ey generally require no further medical or surgical \ntreatment. As such, only a few procedures fulﬁ ll these criteria: \nligation/division/occlusion of a patent ductus arteriosus and \nclosure of an isolated secundum atrial septal defect. Other inter-\nventions or surgical procedures may result in repair or “correc-\ntion,” however not necessarily in normal hemodynam

Chunking

In [22]:
from src.backend.textchunker import chunking

In [23]:
chunks = chunking(texts).chunker()

In [24]:
chunks[1].page_content

'oxygen to part of your heart muscle. \n• \nHeart valve disease: A valve in your heart isn’t working right. \n• \nHigh blood pressure: Your blood is pushing too hard against your artery walls. \n• \nHigh cholesterol: Your blood has too many fats in it. \n• \nPericarditis: Inflammation in your heart’s lining (pericardium). \nCommon signs or symptoms of heart conditions \nSymptoms of heart conditions include: \n• \nChest pain. \n• \nHeart palpitations. \n• \nDizziness. \n• \nShortness of breath. \n• \nFatigue. \n• \nSwelling in your lower body. \nCommon tests to check the health of your heart \nTests to check your heart health include:'

EMbedding and saving in Vector DB

In [25]:

from src.constants import GOOGLE_API_KEY


In [2]:
from src.backend.vectordb_handler import VectorDBManagerOpenAI
vdb = VectorDBManagerOpenAI(
        db_path="./chroma_db",
        collection_name="heart_data_vb"
    )

In [46]:
stored_ids = vdb.store_documents(
        text=text,
        metadatas=metadata
    )

✅ Stored 65 document(s) in vector DB 'heart_data_vb'


In [3]:
vdb.peek(1)

{'ids': ['60a5dbca-a27e-44db-92cd-4fd425405e5a'],
 'embeddings': array([[-3.06150480e-03,  3.08360513e-02, -2.41750441e-02,
         -1.59890223e-02, -2.22453289e-02,  7.69371837e-02,
          2.87633501e-02, -6.42700866e-02, -2.49724258e-02,
          2.21080221e-02, -9.29812319e-04,  2.54862551e-02,
          5.27324416e-02, -3.41078825e-02,  2.00824067e-02,
         -7.33709782e-02,  2.99585573e-02, -4.13971953e-02,
         -4.58694175e-02, -2.77660474e-05,  9.69837047e-03,
         -3.40261273e-02, -1.33345798e-02,  6.59902347e-03,
         -3.75465602e-02, -7.34835267e-02, -1.05055999e-02,
         -7.37927575e-03,  1.10963630e-02, -1.39839919e-02,
         -3.28950286e-02,  1.10182688e-02,  5.96501045e-02,
          1.31544266e-02,  5.18868007e-02, -3.64502668e-02,
         -2.76531801e-02, -4.76227812e-02, -1.02353189e-02,
          2.41137249e-03, -3.40513289e-02, -3.27372737e-02,
         -4.82407659e-02, -1.25363935e-02, -1.31411664e-02,
         -5.81094436e-03,  2.9708048

QUERRYING DB

In [3]:
querry ="Symptoms of heart conditions"
x = vdb.query(query= "Unusual thickening, enlargement or stiffening of your heart muscle, What is this condition calle?")

LLM Response

In [9]:
import google.genai as genai
import os
import dotenv

dotenv.load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
client = genai.Client(api_key = GOOGLE_API_KEY)

# 2️⃣ Define the function
def llm(query, passage):
    prompt = f"Context:\n{passage}\n\nQuestion: {query}"
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
    )
    print(response.text)


In [10]:
querry ="Symptoms of heart conditions give me top 5"

llm(passage= x, query=querry)

The top 5 symptoms of heart conditions are:

1.  Chest pain
2.  Heart palpitations
3.  Dizziness
4.  Shortness of breath
5.  Fatigue


In [5]:
from src.backend.llm import llm
llm_obj = llm()


In [7]:
response = llm_obj.genrate_response(query= querry, context= x)

In [9]:
print(response.text)

Symptoms of heart conditions include:
*   Chest pain
*   Heart palpitations
*   Dizziness
*   Shortness of breath
*   Fatigue
*   Swelling in your lower body


Creating conversation history

In [1]:
%cd "C:\Users\PUNEETH\Desktop\All files\RAG\niyathi_rag"

C:\Users\PUNEETH\Desktop\All files\RAG\niyathi_rag


In [4]:
from langchain.memory import ConversationBufferWindowMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain

# Initialize memory (keeps last 10 messages)
memory = ConversationBufferWindowMemory(
    k=10,
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

# Create conversational chain
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vdb.as_retriever(),
    memory=memory
)

# Use it
# response = qa_chain({"question": user_query})

ModuleNotFoundError: No module named 'langchain.memory'

In [5]:
from collections import deque

class ConversationHistory:
    def __init__(self, max_history=10):
        self.history = deque(maxlen=max_history)
    
    def add_exchange(self, user_query, assistant_response):
        self.history.append({
            "user": user_query,
            "assistant": assistant_response
        })
    
    def get_history_string(self):
        """Format history for LLM context"""
        history_text = ""
        for exchange in self.history:
            history_text += f"User: {exchange['user']}\nAssistant: {exchange['assistant']}\n\n"
        return history_text
    
    def clear(self):
        self.history.clear()

In [ ]:
def llm_with_history(client,query, passage, conversation_history):
    # Get formatted history
    history_context = conversation_history.get_history_string()
    
    # Build prompt with history
    prompt = f"""Previous Conversation:
    {history_context}

    Context (from documents):
    {passage}

    Current Question: {query}

    Please answer the current question, considering the conversation history if relevant."""
    
    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=prompt,
    )
    
    # Add this exchange to history
    conversation_history.add_exchange(query, response.text)
    
    return response

In [ ]:
from src.constants import GOOGLE_API_KEY
# Initialize conversation history
chat_history = ConversationHistory(max_history=10)

from src.backend.vectordb_handler import VectorDBManagerOpenAI
vdb = VectorDBManagerOpenAI(
        db_path="./chroma_db",
        collection_name="heart_data_vb"
)

# Chat loop
client = genai.Client(api_key = GOOGLE_API_KEY)
while True:
    user_query = input("You: ")
    
    if user_query.lower() in ['exit', 'quit']:
        break
    
    # Query vector DB for relevant context
    relevant_docs = vdb.query(query=user_query)
    
    # Generate response with history
    response = llm_with_history(client,user_query, relevant_docs, chat_history)
    
    print(f"Assistant: {response.text}\n")

NameError: name 'client' is not defined